# <center> Adapter <center>
Este notebook explica cómo transformamos las posiciones detectadas por el tracker en un dataframe.

**Adapter** se encarga de:
- Crear un dataframe a partir de las posiciones.
- Separar los valores en las columnas que nos interesan.
- Cambiar el punto de referencia.


#### Inicialización

In [4]:
import pandas as pd
from api.storage import get_keypoints, get_input_video

VIDEO_NAME = "video2"
VIDEO_EXTENSION = "mp4"
video = get_input_video(VIDEO_NAME, VIDEO_EXTENSION)
keypoints_positions = get_keypoints(VIDEO_NAME)
objects = ["shoulder", "elbow", "wrist"]
height = video.get_height()

#### Creación del dataframe

In [5]:
df = pd.DataFrame(keypoints_positions, columns = objects)
df

,shoulder,elbow,wrist
0,"[150, 266]","[119, 438]","[123, 571]"
1,"[151, 266]","[120, 438]","[124, 571]"
2,"[151, 265]","[120, 437]","[124, 569]"
3,"[151, 265]","[120, 437]","[125, 568]"
4,"[150, 265]","[120, 437]","[126, 569]"
...,...,...,...
844,"[124, 273]","[115, 433]","[191, 391]"
845,"[125, 273]","[115, 434]","[191, 393]"
846,"[125, 273]","[115, 434]","[192, 398]"
847,"[125, 273]","[115, 435]","[193, 400]"


Cada valor [x, y] representa la posición de ese objeto en un frame

#### Separación en columnas
Para acceder más facilmente separamos cada [x, y] en dos columns utilizando `pd.Series`

In [6]:
def split_columns(columns: list):
    for column in columns:
        df[[f"rx_{column}", f"ry_{column}"]] = df[column].apply(pd.Series)
        df.drop(columns = [column], inplace = True)

split_columns(objects)
df

,rx_shoulder,ry_shoulder,rx_elbow,ry_elbow,rx_wrist,ry_wrist
0,150,266,119,438,123,571
1,151,266,120,438,124,571
2,151,265,120,437,124,569
3,151,265,120,437,125,568
4,150,265,120,437,126,569
...,...,...,...,...,...,...
844,124,273,115,433,191,391
845,125,273,115,434,191,393
846,125,273,115,434,192,398
847,125,273,115,435,193,400


#### Cambio de punto de referencia
Las posiciones que tenemos son en referencia a la esquina superior izquierda, pero nos resulta más cómodo trabajar con $(0,0)$ en la esquina inferior izquierda.

In [7]:
def move_catesian_origin_to_bottom_left(columns: list):
    for column in columns:
        df[column] = height - df[column]

move_catesian_origin_to_bottom_left([f"ry_{column}" for column in objects])
df

,rx_shoulder,ry_shoulder,rx_elbow,ry_elbow,rx_wrist,ry_wrist
0,150,582,119,410,123,277
1,151,582,120,410,124,277
2,151,583,120,411,124,279
3,151,583,120,411,125,280
4,150,583,120,411,126,279
...,...,...,...,...,...,...
844,124,575,115,415,191,457
845,125,575,115,414,191,455
846,125,575,115,414,192,450
847,125,575,115,413,193,448
